# Setup

### Import necessary modules and do some basic setup.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# TensorFlow ≥2.0 is required
import tensorflow_addons as tfa
import tensorflow_probability as tfp
import tensorflow as tf
assert tf.__version__ >= '2.0'

from tensorflow import keras
from tensorflow.keras import layers

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Common imports
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import dask
import time
import math
import imageio
import PIL
dask.config.set({'array.slicing.split_large_chunks': False})

from IPython import display

# To make this notebook's output stable across runs
np.random.seed(42)

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Custom utils
from utils.utils_data import *
from utils.utils_ml import *
from utils.utils_plot import *

### Define some paths and constants.

In [ ]:
# Paths
DATADIR = os.path.join(os.getcwd(), '..', 'data')

# Some constants
DATE_START = '1979-01-01'
DATE_END = '2020-12-31'
YY_TRAIN = [1979, 2015]
YY_TEST = [2016, 2020]

# Preparing precipitation data

RhiresD daily gridded dataset (MeteoSwiss)

<img src="images/RhiresD.png" alt="RhiresD" height="350"/>

### Load, split and normalize data

In [ ]:
# Load original gridded data
prec = xr.open_dataset(DATADIR + '/MeteoSwiss/RhiresD_ch02.lonlat_19790101_20210731.nc')
prec = prec.sel(time=slice(DATE_START, DATE_END))

In [ ]:
# Split set into (training + validation) and testing based on dates
prec_train_full = prec.sel(time=slice('{}-01-01'.format(YY_TRAIN[0]), '{}-12-31'.format(YY_TRAIN[1])))
prec_test = prec.sel(time=slice('{}-01-01'.format(YY_TEST[0]), '{}-12-31'.format(YY_TEST[1])))

In [ ]:
# Transform to numpy arrays
X_train_full = prec_train_full.RhiresD.to_numpy()
X_test = prec_test.RhiresD.to_numpy()

# Adding the channel dimension
X_train_full = np.expand_dims(X_train_full, axis=3)
X_test = np.expand_dims(X_test, axis=3)

X_train_full.shape

In [ ]:
# Flip latitude axis
X_train_full = np.flip(X_train_full, axis=1)
X_test = np.flip(X_test, axis=1)

In [ ]:
# Cut border to ease use in CNN
X_train_full = X_train_full[:, 0:100, 0:240, :]
X_test = X_test[:, 0:100, 0:240, :]

X_train_full.shape

In [ ]:
# Value/nan mask 
mask = np.isnan(X_train_full[0,:,:,0])

In [ ]:
# Split full training into training and validation sets (and shuffle)
X_train, X_valid = train_test_split(X_train_full, test_size=0.25, random_state=42)

In [ ]:
# Normalize data
X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True)
#X_train = (X_train - X_mean) / X_std
#X_valid = (X_valid - X_mean) / X_std
#X_test = (X_test - X_mean) / X_std


In [ ]:
# Replace nans with 0s
X_train = np.nan_to_num(X_train)
X_valid = np.nan_to_num(X_valid)
X_test = np.nan_to_num(X_test)

In [ ]:
plt.figure(figsize=(8, 6))
img = X_train[100,:,:,0]
img[mask] = np.nan
#cs = ax.contourf(x, y, img, clevs, cmap=cmap, norm=norm)
plt.set_cmap('viridis_r')
plt.imshow(img)
plt.axis('off')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(8, 6))
img = X_mean.squeeze(axis=3).squeeze(axis=0)
plt.imshow(img)
plt.axis('off')
plt.tight_layout()
plt.title('Mean precipitation')

plt.figure(figsize=(8, 6))
img = X_std.squeeze(axis=3).squeeze(axis=0)
plt.imshow(img)
plt.axis('off')
plt.tight_layout()
plt.title('Std of precipitation')

In [ ]:
# Transform to Tensorflow dataset
batch_size = 32
train_dataset = (tf.data.Dataset.from_tensor_slices(X_train).batch(batch_size))
test_dataset = (tf.data.Dataset.from_tensor_slices(X_valid).batch(batch_size))

# Convolutional Autoencoder

### Define the model

In [ ]:
class CAE(tf.keras.Model):
    """Convolutional autoencoder."""

    def __init__(self, latent_dim=2, x_size=28, y_size=28):
        super(CAE, self).__init__()
        self.x_size = x_size
        self.y_size = y_size
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential(
            [
                layers.InputLayer(
                    input_shape=(self.x_size, self.y_size, 1)),
                layers.Conv2D(
                    filters=32, 
                    kernel_size=3, 
                    strides=(2, 2), 
                    activation='relu'),
                layers.Conv2D(
                    filters=64, 
                    kernel_size=3, 
                    strides=(2, 2), 
                    activation='relu'),
                layers.Flatten(),
                layers.Dense(latent_dim, activation='relu'),
            ]
        )

        self.decoder = tf.keras.Sequential(
            [
                layers.InputLayer(
                    input_shape=(latent_dim,)),
                layers.Dense(
                    units=(int(self.x_size/4))*int((self.y_size/4))*32, 
                    activation='relu'),
                layers.Reshape(
                    target_shape=(int(self.x_size/4), int(self.y_size/4), 32)),
                layers.Conv2DTranspose(
                    filters=64, 
                    kernel_size=3, 
                    strides=2, 
                    padding='same',
                    activation='relu'),
                layers.Conv2DTranspose(
                    filters=32, 
                    kernel_size=3, 
                    strides=2, 
                    padding='same',
                    activation='relu'),
                layers.Conv2DTranspose(
                    filters=1, 
                    kernel_size=3, 
                    strides=1, 
                    padding='same'),
            ]
        )
    
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [ ]:
autoencoder = CAE(latent_dim=2, x_size=100, y_size=240)


autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(X_test, X_test,
                epochs=30,
                #shuffle=True,
                validation_data=(X_valid, X_valid))

In [ ]:
encoded_imgs = autoencoder.encoder(X_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

In [ ]:
n = 4
plt.figure(figsize=(40, 8))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    img = X_test[i]
    img[mask] = np.nan
    plt.imshow(img)
    plt.title("original")
    #plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    img = decoded_imgs[i]
    img[mask] = np.nan
    plt.imshow(img)
    plt.title("reconstructed")
    #plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()


---

# Convolutional Variational Autoencoder

### Define the model
Architecture from Tensorflow tutorials (https://www.tensorflow.org/tutorials/generative/cvae)

In [ ]:
class CVAE(tf.keras.Model):
    """Convolutional variational autoencoder."""

    def __init__(self, latent_dim):
        super(CVAE, self).__init__()
        self.x_size = 100
        self.y_size = 240
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(
                    input_shape=(self.x_size, self.y_size, 1)),
                tf.keras.layers.Conv2D(
                    filters=32, 
                    kernel_size=3, 
                    strides=(2, 2), 
                    activation='relu'),
                tf.keras.layers.Conv2D(
                    filters=64, 
                    kernel_size=3, 
                    strides=(2, 2), 
                    activation='relu'),
                tf.keras.layers.Flatten(),
                # No activation
                tf.keras.layers.Dense(latent_dim + latent_dim),
            ]
        )

        self.decoder = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(
                    input_shape=(latent_dim,)),
                tf.keras.layers.Dense(
                    units=(int(self.x_size/4))*int((self.y_size/4))*32, 
                    activation=tf.nn.relu),
                tf.keras.layers.Reshape(
                    target_shape=(int(self.x_size/4), int(self.y_size/4), 32)),
                tf.keras.layers.Conv2DTranspose(
                    filters=64, 
                    kernel_size=3, 
                    strides=2, 
                    padding='same',
                    activation='relu'),
                tf.keras.layers.Conv2DTranspose(
                    filters=32, 
                    kernel_size=3, 
                    strides=2, 
                    padding='same',
                    activation='relu'),
                # No activation
                tf.keras.layers.Conv2DTranspose(
                    filters=1, 
                    kernel_size=3, 
                    strides=1, 
                    padding='same'),
            ]
        )

    @tf.function
    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(100, self.latent_dim))
        return self.decode(eps, apply_sigmoid=False) # True

    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean

    def decode(self, z, apply_sigmoid=False):
        logits = self.decoder(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits


### Train the model
Code from Tensorflow tutorials (https://www.tensorflow.org/tutorials/generative/cvae)

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4)


def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    return tf.reduce_sum(
        -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
        axis=raxis)


def compute_loss(model, x):
    mean, logvar = model.encode(x)
    z = model.reparameterize(mean, logvar)
    
    x_logit = model.decode(z)
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(
        logits=x_logit, labels=x)
    print(cross_ent.shape)
    #logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
    
    x_pred = model.decode(z)
    print(x_pred.shape)
    mse = tf.keras.metrics.mean_squared_error(x, x_pred)
    print(mse.shape)
    logpx_z = -tf.reduce_sum(mse, axis=[1, 2, 3])
    print(logpx_z.shape)
    
    logpz = log_normal_pdf(z, 0., 0.)
    logqz_x = log_normal_pdf(z, mean, logvar)
    return -tf.reduce_mean(logpx_z + logpz - logqz_x)


@tf.function
def train_step(model, x, optimizer):
    """Executes one training step and returns the loss.

    This function computes the loss and gradients, and uses the latter to
    update the model's parameters.
    """
    with tf.GradientTape() as tape:
        loss = compute_loss(model, x)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))


In [ ]:
epochs = 10
# set the dimensionality of the latent space to a plane for visualization later
latent_dim = 2
num_examples_to_generate = 16

# keeping the random vector constant for generation (prediction) so
# it will be easier to see the improvement.
random_vector_for_generation = tf.random.normal(
    shape=[num_examples_to_generate, latent_dim])
model = CVAE(latent_dim)


In [ ]:
def generate_and_save_images(model, epoch, test_sample):
    mean, logvar = model.encode(test_sample)
    z = model.reparameterize(mean, logvar)
    predictions = model.sample(z)
    fig = plt.figure(figsize=(20, 15))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i, :, :, 0], cmap='gray')
        plt.axis('off')

    # tight_layout minimizes the overlap between 2 sub-plots
    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()


In [ ]:
# Pick a sample of the test set for generating output images
assert batch_size >= num_examples_to_generate
for test_batch in test_dataset.take(1):
    test_sample = test_batch[0:num_examples_to_generate, :, :, :]


In [ ]:
generate_and_save_images(model, 0, test_sample)

for epoch in range(1, epochs + 1):
    start_time = time.time()
    for train_x in train_dataset:
        train_step(model, train_x, optimizer)
    end_time = time.time()

    loss = tf.keras.metrics.Mean()
    for test_x in test_dataset:
        loss(compute_loss(model, test_x))
    elbo = -loss.result()
    display.clear_output(wait=False)
    print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
          .format(epoch, elbo, end_time - start_time))
    generate_and_save_images(model, epoch, test_sample)


In [ ]:
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
plt.imshow(display_image(epoch))
plt.axis('off')  # Display images

In [ ]:
anim_file = 'cvae.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)

In [ ]:
def plot_latent_images(model, n, digit_size=28):
  """Plots n x n digit images decoded from the latent space."""

  norm = tfp.distributions.Normal(0, 1)
  grid_x = norm.quantile(np.linspace(0.05, 0.95, n))
  grid_y = norm.quantile(np.linspace(0.05, 0.95, n))
  image_width = digit_size*n
  image_height = image_width
  image = np.zeros((image_height, image_width))

  for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
      z = np.array([[xi, yi]])
      x_decoded = model.sample(z)
      digit = tf.reshape(x_decoded[0], (digit_size, digit_size))
      image[i * digit_size: (i + 1) * digit_size,
            j * digit_size: (j + 1) * digit_size] = digit.numpy()

  plt.figure(figsize=(20, 15))
  plt.imshow(image, cmap='Greys_r')
  plt.axis('Off')
  plt.show()

In [ ]:
plot_latent_images(model, 20)